In [18]:
import pandas as pd
import numpy as np

from scipy.sparse import coo_matrix

import implicit

Читаем клики пользователей

In [19]:
train_clicks = pd.read_csv('./raw_data/train_clicks.csv')

In [20]:
users = train_clicks.user_id
items = train_clicks.picture_id

Создаём разреженную матрицу пользователь — объект

In [21]:
user_item = coo_matrix((np.ones_like(users), (users, items)))

В качестве модели будем использовать разложение матрицы с помощью метода ALS

In [22]:
np.random.seed(42)
model = implicit.als.AlternatingLeastSquares(factors=10, iterations=30, use_gpu=False)

In [23]:
model.fit(user_item.T.tocsr())

100%|██████████| 30.0/30 [03:53<00:00,  7.34s/it]


Прочитаем идентификаторы пользователей, для которых нам нужно сделать предсказания

In [24]:
test_users = pd.read_csv('raw_data/test_users.csv')

In [25]:
user_item_csr = user_item.tocsr()

Для каждого пользователя найдём 100 самых релевантных изображений

In [26]:
rows = []
for user_id in test_users.user_id.values:
    items = [i[0] for i in model.recommend(user_id, user_item_csr, N=100)]
    rows.append(items)

Отформатируем идентификаторы как нужно для ответа

In [27]:
test_users['predictions'] = list(map(lambda x: ' '.join(map(str, x)), rows))

И запишем предсказания в файл

In [28]:
test_users.to_csv('baseline_predictions.csv', index=False)